In [ ]:
# Preperare distributed
using Distributed
addprocs(Sys.CPU_THREADS - nprocs() - 1)
print("Number of workers: ", nprocs(), "\nNumber of CPU threads: ", Sys.CPU_THREADS, "\n")

# Optimization and benchmarking
using BenchmarkTools

# Wave equation helpers
include("helpers/wave_equations.jl")

# Plotting
@everywhere using Plots
include("helpers/distributed_gif.jl")

# Problem 1.1

In [ ]:
# Setup for benchmarking
N_test = 100000
psi_tst = rand(N_test)
do_bench = true

In [ ]:
if do_bench
    @benchmark wave_equation(psi_tst, 1.0, 1.0, N_test)
end

In [ ]:
if do_bench
    @benchmark wave_equation_inb(psi_tst, 1.0, 1.0, N_test)
end

In [ ]:
if do_bench
    @benchmark wave_equation_vec(psi_tst, 1.0, 1.0, N_test)
end

In [ ]:
if do_bench
    @benchmark wave_equation_dist(psi_tst, 1.0, 1.0, N_test)
end

In [ ]:
if do_bench
    @benchmark wave_equation_simd(psi_tst, 1.0, 1.0, N_test)
end

In [ ]:
if do_bench
    @benchmark wave_equation_avx(psi_tst, 1.0, 1.0, N_test)
end

In [ ]:
if do_bench
    @benchmark wave_equation_dist_avx(psi_tst, 1.0, 1.0, N_test)
end

In [ ]:
initial_condition_0 = (x) -> sin( x)
initial_condition_1 = (x) -> sin(2pi * x)
initial_condition_2 = (x) -> sin(5pi * x)
initial_condition_3 = (x) -> (x > 1/5 && x < 2/5) ? sin(5pi * x) : 0.0

In [ ]:
# Test finite difference approximation of the wave equation
L = 2pi
N = 100
dx = L/N
x = 0:dx:L
psi = initial_condition_0.(x)
dpsi_dt = zeros(N)
c = 1.0
d2psi_dt2 = wave_equation_avx(psi, c, L, N)
# Plot initial condition and second time derivative
plot(x, psi, label="Initial Condition", title="Wave Equation Test")
plot!(x[2:end-1], d2psi_dt2, label="Second Time Derivative", xlabel="x", ylabel="Value")

In [ ]:
t_0 = 0.0
t_f = 1.0
dt = 0.001
c=1.0
L=1.0
N=1000

## Initial conditions

In [ ]:
solution_1_euler = propagate_psi(initial_condition_1, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)
solution_2_euler = propagate_psi(initial_condition_2, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)
solution_3_euler = propagate_psi(initial_condition_3, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)

In [ ]:
function get_plots(t_f::Float64, t_0::Float64, dt::Float64, L::Float64, N::Int, solution_1::Matrix, solution_2::Matrix, solution_3::Matrix):: Vector{Plots.Plot{Plots.GRBackend}}
    x = range(0, L, length=N)
    i_total = Int((t_f - t_0) / dt)
    plots = @distributed (vcat) for i in 1:i_total
        # create a plot with 3 subplots and a custom layout
        p = plot(x, solution_1[:, i], ylim=(-1, 1), title="Time: $(round(i*dt, digits=2)) s", xlabel="Position along string", ylabel="Displacement")
        plot!(x, solution_2[:, i])
        plot!(x, solution_3[:, i])
    end
    return plots
end

In [ ]:
plots = get_plots(t_f, t_0, dt, L, N, solution_1_euler, solution_2_euler, solution_3_euler)

In [ ]:
gif_slow(plots, "ex_1_anim_euler_slow.gif")

In [ ]:
distributed_gif(plots, "ex_1_anim_euler.gif"; do_palette=false)

In [ ]:
solution_1_leapfrog = propagate_psi_leapfrog(initial_condition_1, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)
solution_2_leapfrog = propagate_psi_leapfrog(initial_condition_2, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)
solution_3_leapfrog = propagate_psi_leapfrog(initial_condition_3, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)
plots_leapfrog = get_plots(t_f, t_0, dt, L, N, solution_1_leapfrog, solution_2_leapfrog, solution_3_leapfrog)

In [ ]:
distributed_gif(plots_leapfrog, "ex_1_anim_leapfrog.gif")

We see visibly smaller 'zig-zag' behaviour in displacement between the peaks of y3